# Warning : 
# Do "File -> Save a copy in Drive" before you start modifying the notebook, otherwise your modifications will not be saved.


Avant de commencer le TP, vérifiez que vous êtes sur un environnement GPU et python 3 : 

Exécution -> Modifier le type d'exécution -> Type d'exécution = python2, Accélerateur matériel = GPU

In [ ]:
from PIL import Image

import numpy as np
import torchvision
import pickle

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from sklearn.svm import LinearSVC

# Partie 1 : Architecture VGG16

In [ ]:
#!wget https://github.com/cdancette/deep-learning-polytech-tp6-7/raw/master/tp8/imagenet_classes.pkl
!wget https://github.com/rdfia/rdfia.github.io/raw/master/data/3-a/imagenet_classes.pkl

# Bonus : Classifiez des exemples avec vgg16 et commentez le résultat dans votre rapport.
!wget --content-disposition https://unsplash.com/photos/gKXKBY-C-Dk/download?force=true -O cat.jpg
!wget --content-disposition https://unsplash.com/photos/qO-PIF84Vxg/download?force=true -O dog.jpg

In [ ]:
cat = Image.open("cat.jpg")
plt.imshow(cat)

# dog = Image.open('dog.jpg')
# plt.imshow(dog)

# Ajoutez vos images
# VOTRE CODE ICI pour le bonus:

vgg16 = torchvision.models.vgg16(pretrained=True)

imagenet_classes = pickle.load(
    open("imagenet_classes.pkl", "rb")
)  # chargement des classes
img = Image.open("cat.jpg")
img = img.resize((224, 224), Image.BILINEAR)
img = np.array(img, dtype=np.float32) / 255
img = img.transpose((2, 0, 1))

img = np.expand_dims(img, 0)
x = torch.Tensor(img)
transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),
    ]
)
x = transform(x)
y = vgg16(x).detach()  # TODO calcul forward
y = y.numpy()  # transformation en array numpy

indices_top_5 = [
    i[0] for i in sorted(enumerate(y[0]), key=lambda k: k[1], reverse=True)
]
indices_top_5 = indices_top_5[:5]

print("   Top 5:")

for idx in indices_top_5:
    print("       ", imagenet_classes[idx], ":", y[0][idx])

In [ ]:
print(imagenet_classes)

In [ ]:
model = vgg16.features
map = model[0](x)
cat = Image.open("cat.jpg")

for i in range(map.shape[1]):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    ax1.imshow(map.detach().numpy()[0][i], cmap="gray")
    ax1.title.set_text("Activation map n°" + str(i + 1) + "/" + str(map.shape[1]))
    ax2.imshow(cat, cmap="gray")
    ax2.title.set_text("Original image")
    plt.show()

**Test with an image from the COCO dataset:**

In [ ]:
img = Image.open("coco.jpg")
plt.figure(figsize=(12, 20))
plt.imshow(img)
plt.show()

img = img.resize((224, 224), Image.BILINEAR)
img = np.array(img, dtype=np.float32) / 255
img = img.transpose((2, 0, 1))
img = np.expand_dims(img, 0)

x = torch.Tensor(img)
x = transform(x)
y = vgg16(x).detach()  # TODO calcul forward
y = y.numpy()  # transformation en array numpy

indices_top_15 = [
    i[0] for i in sorted(enumerate(y[0]), key=lambda k: k[1], reverse=True)
]
indices_top_15 = indices_top_15[:15]

print("   Top 15:")

for idx in indices_top_15:
    print("       ", imagenet_classes[idx], ":", y[0][idx])

**Test with an image of a brown bear, which is a class present in ImageNet:**

In [ ]:
img = Image.open("brown_bear.png")
plt.figure(figsize=(12, 20))
plt.imshow(img)
plt.show()

img = img.resize((224, 224), Image.BILINEAR)
img = np.array(img, dtype=np.float32) / 255
img = img.transpose((2, 0, 1))
img = np.expand_dims(img, 0)

x = torch.Tensor(img)

x = transform(x)
y = vgg16(x).detach()  # TODO calcul forward
y = y.numpy()  # transformation en array numpy

indices_top_15 = [
    i[0] for i in sorted(enumerate(y[0]), key=lambda k: k[1], reverse=True)
]
indices_top_15 = indices_top_15[:15]

print("   Top 15:")

for idx in indices_top_15:
    print("       ", imagenet_classes[idx], ":", y[0][idx])

**Test with an image of a duck which is a class present in ImageNet:**

In [ ]:
img = Image.open("concept_drift_small.png")
plt.figure(figsize=(12, 20))
plt.imshow(img)
plt.show()

img = img.resize((224, 224), Image.BILINEAR)
img = np.array(img, dtype=np.float32) / 255
img = img.transpose((2, 0, 1))
img = np.expand_dims(img, 0)

x = torch.Tensor(img)

x = transform(x)
y = vgg16(x).detach()  # TODO calcul forward
y = y.numpy()  # transformation en array numpy

indices_top_15 = [
    i[0] for i in sorted(enumerate(y[0]), key=lambda k: k[1], reverse=True)
]
indices_top_15 = indices_top_15[:15]

print("   Top 15:")

for idx in indices_top_15:
    print("       ", imagenet_classes[idx], ":", y[0][idx])

# Partie 2: Transfer Learning avec VGG16 sur 15 Scene

In [ ]:
#!wget https://github.com/cdancette/deep-learning-polytech-tp6-7/raw/master/tp8/15ScenesData.zip
!wget https://github.com/rdfia/rdfia.github.io/raw/master/data/3-a/15ScenesData.zip

!unzip 15ScenesData.zip

In [ ]:
!ls 15SceneData/test/bedroom/    

In [ ]:
from torchsummary import summary

In [ ]:
class VGG16relu7(nn.Module):
  def __init__(self,layer_to_stop_at):
    super(VGG16relu7, self).__init__()

    # Copy the entire convolutional part
    self.features = nn.Sequential( *list(vgg16.features.children())[:len(list(vgg16.features.children()))+3-layer_to_stop_at-1])

    #summary(self.features, (3,224,224))
    
    # Keep a piece of the classifier: -2 to stop at relu7s
    if(layer_to_stop_at in [1,2]):
      self.classifier = nn.Sequential(*list(vgg16.classifier.children())[:-layer_to_stop_at])
    elif(layer_to_stop_at == 0):
      self.classifier = nn.Sequential(*list(vgg16.classifier.children()))     

    self.layer_to_stop_at = layer_to_stop_at

  def forward(self, x):
    x = self.features(x)
    x = x.view(x.size(0), -1)
    
    if(self.layer_to_stop_at in [0,1,2]):
      x = self.classifier(x)

    return x

In [ ]:
CUDA = True

def get_dataset(batch_size, path):
    # Cette fonction permet de recopier 3 fois une image qui
    # ne serait que sur 1 channel (donc image niveau de gris)
    # pour la "transformer" en image RGB. Utilisez la avec
    # transform.Lambda
    def duplicateChannel(img):
        img = img.convert("L")
        np_img = np.array(img, dtype=np.uint8)
        np_img = np.dstack([np_img, np_img, np_img])
        img = Image.fromarray(np_img, "RGB")
        return img

    #####################
    ## Votre code ici  ##
    #####################
    # Ajouter le pré-traitement
    train_dataset = datasets.ImageFolder(
        path + "/train",
        transform=transforms.Compose(
            [  # Pré-traitement à faire
                transforms.Lambda(duplicateChannel),
                transforms.ToTensor(),
                transforms.Resize((224, 224)),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        ),
    )
    val_dataset = datasets.ImageFolder(
        path + "/test",
        transform=transforms.Compose(
            [  # Pré-traitement à faire
                transforms.Lambda(duplicateChannel),
                transforms.ToTensor(),
                transforms.Resize((224, 224)),
                transforms.Normalize(
                    mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
                ),
            ]
        ),
    )
    ####################
    ##      FIN        #
    ####################

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=False,
        pin_memory=CUDA,
        num_workers=2,
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        pin_memory=CUDA,
        num_workers=2,
    )

    return train_loader, val_loader

In [ ]:
PRINT_INTERVAL = 50


def extract_features(data, model):
    #####################
    ## Votre code ici  ##
    #####################
    # init features matrices
    y = np.empty((0))
    ####################
    ##      FIN        #
    ####################

    for i, (input, target) in enumerate(data):
        if i % PRINT_INTERVAL == 0:
            print("Batch {0:03d}/{1:03d}".format(i, len(data)))
        if CUDA:
            input = input.cuda()
        #####################
        ## Votre code ici  ##
        #####################
        # Feature extraction à faire
        features = model(input).detach().cpu().numpy()
        features = features / np.linalg.norm(features, 2, 1, True)

        if i == 0:
            X = features
        else:
            X = np.vstack((X, features))

        y = np.append(y, target.numpy())
        ####################
        ##      FIN        #
        ####################
    print("size of the extracted features:", X.shape)

    return X, y


def search_hyperparameters(C_values, X_train, y_train, X_valid, y_valid):
    accuracies = []

    for c in C_values:
        print(f"Training SVM with C={c}")
        clf = LinearSVC(C=c)

        # Fit on sub-train set
        clf.fit(X_train, y_train)

        # Evaluate on val set
        acc = clf.score(X_valid, y_valid)
        accuracies.append(acc)
        print(f"--> accuracy = {acc}")
    return accuracies


def plot_accuracies(C_values, X_train, y_train, X_valid, y_valid):
    plt.figure(figsize=(12, 7))
    accuracies = search_hyperparameters(C_values, X_train, y_train, X_valid, y_valid)
    plt.title(f"kernel : linear", fontsize=18)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.xlabel("C", fontsize=18)
    plt.ylabel("Accuracy", fontsize=18)
    plt.plot(C_values, accuracies)

    print(
        "\nBest C =",
        C_values[np.argmax(accuracies)],
        "  Best accuracy =",
        np.max(accuracies),
    )

    return accuracies


def SVM_on_extracted_features(
    path="15SceneData", batch_size=8, layer_to_stop_at=0, model=VGG16relu7
):
    #####################
    ## Votre code ici  ##
    #####################
    # Remplacer par le modèle par un réseau tronqué pour faire de la feature extraction
    # On créera une nouvelle classe VGG16relu7 ici
    model = model(layer_to_stop_at)
    ####################
    ##      FIN        #
    ####################
    model.eval()
    if CUDA:  # si on fait du GPU, passage en CUDA
        cudnn.benchmark = True
        model = model.cuda()

    # On récupère les données
    print("Récupération des données")
    train, test = get_dataset(batch_size, path)

    # Extraction des features
    print("Feature extraction")
    X_train, y_train = extract_features(train, model)
    X_test, y_test = extract_features(test, model)

    #####################
    ## Votre code ici  ##
    #####################
    # Apprentissage et évaluation des SVM à faire
    print("Apprentissage des SVM")
    clf = LinearSVC(C=1)

    # Fit on sub-train set
    clf.fit(X_train, y_train)

    # Evaluate on val set
    accuracy = clf.score(X_test, y_test)

    ####################
    ##      FIN        #
    ####################
    print("\n[C = 1] Accuracy = %f" % accuracy, "\n")

    # TUNING OF THE C PARAMETER ================================================
    C_values = np.arange(0.1, 3, 0.05)
    accuracies = plot_accuracies(C_values, X_train, y_train, X_test, y_test)
    # ==========================================================================

In [ ]:
SVM_on_extracted_features(
    "15SceneData", batch_size=8, layer_to_stop_at=0
)  # 1st layer from the end

In [ ]:
SVM_on_extracted_features(
    "15SceneData", batch_size=8, layer_to_stop_at=1
)  # 2nd layer from the end

In [ ]:
SVM_on_extracted_features(
    "15SceneData", batch_size=8, layer_to_stop_at=2
)  # 3rd layer from the end

In [ ]:
SVM_on_extracted_features(
    "15SceneData", batch_size=8, layer_to_stop_at=3
)  # 4th layer from the end

In [ ]:
SVM_on_extracted_features(
    "15SceneData", batch_size=8, layer_to_stop_at=4
)  # 5th layer from the end

In [ ]:
SVM_on_extracted_features(
    "15SceneData", batch_size=8, layer_to_stop_at=8
)  # 5th layer from the end

In [ ]:
resnet50 = torchvision.models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
print(resnet50)
print(vgg16)

In [ ]:
class resnet50(nn.Module):
    def __init__(self, layer_to_stop_at):
        super(resnet50, self).__init__()

        # Copy the entire convolutional part
        self.features = nn.Sequential(*list(resnet50.children())[0:8])
        self.classifier = nn.Sequential(*list(resnet50.children())[9])

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)

        # x = self.classifier(x)

        return x

In [ ]:
SVM_on_extracted_features("15SceneData", batch_size=8, layer_to_stop_at=0)